In [ ]:
from __future__ import print_function
import torch
from torch import nn
from torch import optim
from torch.utils.data import DataLoader, random_split
from torchvision import datasets, models, transforms
import os
import numpy as np
import sys

from torchvision.datasets import VisionDataset
from torchvision.datasets.utils import check_integrity #, download_and_extract_archive
import pandas
import csv
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import confusion_matrix
import time
import copy

if torch.cuda.is_available():
    device = torch.device('cuda:0')
else:
    device = torch.device('cpu')

In [ ]:
%load_ext autoreload

sys.path.append(os.path.expanduser("~/few-shot-learning/"))
from few_shot.datasets import FashionProductImages

In [ ]:
X, y = fashion[10]
# X, y = fashion[10:15] # fails

len(fashion)

for i in range(len(fashion)):
    X, y = fashion[i]
    # print(y.shape)
    # if not isinstance(y, int):
    #    import pdb; pdb.set_trace()
    if not (X.shape[1]==80 and X.shape[2]==60):
        import pdb; pdb.set_trace()

counter = 0
for (i, batch) in enumerate(train_loader):
    counter += 64
    print(counter)
    # import pdb; pdb.set_trace()
    
    

In [ ]:
class_sample_count = np.bincount(fashion.target_indices, minlength=fashion.n_classes)
print(class_sample_count)
print(len(class_sample_count))
print(sum(class_sample_count))

print(np.unique(fashion.targets))
print(len(np.unique(fashion.targets)))

print(np.unique(fashion.df_meta["articleType"]))
print(len(np.unique(fashion.df_meta["articleType"])))

# all perfumes are from 2017, which means they're all in the test set
fashion.df_meta[
    (fashion.df_meta["articleType"]=="Perfume and Body Mist")
     & (fashion.df_meta["year"] == 2017.0)
]

In [ ]:
datasets = {
    classes: {
        split: FashionProductImages(
            "~/data",
            split=split,
            classes=classes,
            transform=data_transforms[split]
        ) for split in ["train", "test"]
    } for classes in ["top", "bottom"]
}

In [ ]:
datasets

In [ ]:
data_transforms = {
    'train': transforms.Compose([
        transforms.RandomResizedCrop((80,60), scale=(0.8, 1.0)),
        # transforms.Resize((80,60)),
        transforms.RandomRotation(degrees=15),
        transforms.ColorJitter(),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
    'val': transforms.Compose([
        transforms.Resize((80,60)),
        # transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
    'test': transforms.Compose([
        transforms.Resize((80,60)),
        # transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
}

fashion = FashionProductImages("~/data", classes="top", split="train", transform=data_transforms["train"])

def prepare_dataset_weights(labels, n_classes):
    # prepare weighted sampling
    class_sample_count = np.bincount(labels, minlength=n_classes)
    class_sample_count = torch.from_numpy(class_sample_count).float()

    class_weights = torch.zeros_like(class_sample_count)
    class_weights[class_sample_count > 0] = 1./class_sample_count[class_sample_count > 0]

    dataset_weights = torch.zeros(len(labels))
    dataset_weights = class_weights[labels]
    
    return dataset_weights

batch_size = 64

# split into training and validation set
train_size = int(len(fashion) * 0.9)

trainset, valset = random_split(fashion, [train_size, len(fashion) - train_size])

training_targets = trainset.dataset.target_indices[trainset.indices]

training_weights = prepare_dataset_weights(training_targets, trainset.dataset.n_classes)
training_sampler = torch.utils.data.WeightedRandomSampler(training_weights, train_size, replacement=True)

train_loader = DataLoader(trainset, batch_size=batch_size, num_workers=4, sampler=training_sampler)
val_loader = DataLoader(valset, batch_size=batch_size, num_workers=4)

dataloaders = {"train": train_loader, "val": val_loader}
dataset_sizes = {"train": len(trainset), "val": len(valset)}

# data_loader = DataLoader(fashion, batch_size=128, shuffle=True, num_workers=4)

In [ ]:
y_counts = np.zeros_like(class_sample_count)

for batch in train_loader:
    X, y = batch
    y_counts += np.bincount(y, minlength=20)
    
print(y_counts)
print(y_counts.sum())

y_counts = np.zeros_like(class_sample_count)

for batch in val_loader:
    X, y = batch
    y_counts += np.bincount(y, minlength=20)
    
print(y_counts)
print(y_counts.sum())

In [ ]:
def train_model(model, criterion, optimizer, scheduler, dataloaders, dataset_sizes, num_epochs=25):
    since = time.time()

    best_model_wts = copy.deepcopy(model.state_dict())
    best_acc = 0.0

    for epoch in range(num_epochs):
        print('Epoch {}/{}'.format(epoch + 1, num_epochs))
        print('-' * 10)

        # Each epoch has a training and validation phase
        for phase in ['train', 'val']:
            if phase == 'train':
                model.train()  # Set model to training mode
            else:
                model.eval()   # Set model to evaluate mode

            running_loss = 0.0
            running_corrects = 0
            # TODO.refactor
            running_confusion_matrix = np.zeros([model.fc.out_features]*2)

            # Iterate over data.
            for inputs, labels in dataloaders[phase]:
                inputs = inputs.to(device)
                labels = labels.to(device)

                # zero the parameter gradients
                optimizer.zero_grad()

                # forward
                # track history if only in train
                with torch.set_grad_enabled(phase == 'train'):
                    outputs = model(inputs)
                    _, preds = torch.max(outputs, 1)
                    loss = criterion(outputs, labels)

                    # backward + optimize only if in training phase
                    if phase == 'train':
                        loss.backward()
                        optimizer.step()

                # statistics
                # multiply batch loss with batch_size to go from mean to sum
                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)
                running_confusion_matrix += confusion_matrix(labels.cpu(), preds.cpu(), labels=np.arange(20))
            if phase == 'train':
                scheduler.step()

            epoch_loss = running_loss / dataset_sizes[phase]
            epoch_acc = running_corrects.double() / dataset_sizes[phase]
            epoch_conf_matrix = running_confusion_matrix.copy()

            print('{} Loss: {:.4f} Acc: {:.4f}'.format(
                phase, epoch_loss, epoch_acc))

            # deep copy the model
            if phase == 'val' and epoch_acc > best_acc:
                best_acc = epoch_acc
                best_model_wts = copy.deepcopy(model.state_dict())
                best_conf_matrix = epoch_conf_matrix.copy()

        print()

    time_elapsed = time.time() - since
    print('Training complete in {:.0f}m {:.0f}s'.format(
        time_elapsed // 60, time_elapsed % 60))
    print('Best val Acc: {:4f}'.format(best_acc))

    # load best model weights
    model.load_state_dict(best_model_wts)
    
    return model, best_conf_matrix

In [ ]:
def visualize_model(model, num_images=6):
    was_training = model.training
    model.eval()
    images_so_far = 0
    fig = plt.figure()

    with torch.no_grad():
        for i, (inputs, labels) in enumerate(dataloaders['val']):
            inputs = inputs.to(device)
            labels = labels.to(device)

            outputs = model(inputs)
            _, preds = torch.max(outputs, 1)

            for j in range(inputs.size()[0]):
                images_so_far += 1
                ax = plt.subplot(num_images//2, 2, images_so_far)
                ax.axis('off')
                ax.set_title('predicted: {}'.format(class_names[preds[j]]))
                imshow(inputs.cpu().data[j])

                if images_so_far == num_images:
                    model.train(mode=was_training)
                    return
        model.train(mode=was_training)

In [ ]:
model_ft = models.resnet18(pretrained=True)
# model_ft = models.resnet50(pretrained=True)

num_ftrs = model_ft.fc.in_features
# Alternatively: nn.Linear(num_ftrs, len(class_names)).
model_ft.fc = nn.Linear(num_ftrs, fashion.n_classes)

model_ft = model_ft.to(device)

criterion = nn.CrossEntropyLoss()

# Observe that all parameters are being optimized
optimizer_ft = optim.Adam(model_ft.parameters(), lr=0.001, betas=(0.9, 0.999))

# Decay LR by a factor of 0.1 every 7 epochs
exp_lr_scheduler = optim.lr_scheduler.StepLR(optimizer_ft, step_size=5, gamma=0.5)

In [ ]:
model_ft, conf_matrix = train_model(
    model_ft, criterion, optimizer_ft, exp_lr_scheduler,
    dataloaders, dataset_sizes,
    num_epochs=50
)

In [ ]:
conf_matrix.sum()